In [2]:
import os

from dotenv import load_dotenv
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentType, initialize_agent, load_tools

### Build a LLM application with LangChain Framework.

#### 1. Set the environment variables.

In [3]:
load_dotenv()

os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_SEARCH_API_KEY")

#### 2. Use the LLM model directly.
Instantiate a chat model and invoke it with the messages.

Reference: https://python.langchain.com/v0.2/docs/tutorials/llm_chain/

In [4]:
model = AzureChatOpenAI(deployment_name="gpt-35-turbo-120")

In [5]:
user_prompt = "請問「奧運」的英文全名是什麼？回答時使用繁體中文和台灣詞語解釋。"

prompts = [HumanMessage(content=user_prompt)]
completion = model.invoke(prompts)
completion.content

'「奧運」的英文全名是「Olympic Games」。奧運是指一個國際性的體育盛事，由國際奧林匹克委員會（International Olympic Committee，縮寫為IOC）主辦。奧運是每四年舉辦一次的大型運動會，各國選手會聚集在一個指定的城市，參加各種體育項目的比賽。奧運的目的是促進全球體育發展、增進國際間的友誼和合作，並鼓勵運動員追求卓越和友誼精神。'

In [7]:
model.predict(user_prompt)

'奧運的英文全名是「Olympic Games」。其中「Olympic」指的是古希臘的奧林匹克，是指奧林匹克神廟的地方，也是古代希臘人舉辦各項競技賽事的場地。而「Games」則代表比賽或運動競賽。因此，「Olympic Games」即指的是以奧林匹克為背景的運動競賽盛會。'

#### 3. [Prompts] Add prompt templates.
Use to create flexible templated prompts for chat models.

Reference: https://python.langchain.com/v0.2/docs/how_to/#prompt-templates

- **String Prompt Templates**  
  Reference: https://python.langchain.com/v0.2/docs/concepts/#string-prompttemplates

In [8]:
system_prompt = "回答時使用繁體中文和台灣詞語解釋。"
user_prompt = "請問「奧運」的英文全名是什麼？"

prompt_template = PromptTemplate.from_template("{question}" + system_prompt)
prompt_template.invoke({"question": user_prompt})
completion = model.invoke(prompts)
completion.content

'「奧運」的英文全名是「Olympic Games」。奧運是指由國際奧林匹克委員會（International Olympic Committee）所舉辦的國際性運動盛會，每四年舉行一次。這項全球盛事吸引來自不同國家和地區的運動員們參與各項體育競賽，包括田徑、游泳、籃球、足球等等。奧運強調競技精神、友誼和國際合作，是體育界最高榮譽的賽事之一。'

- **Chat Prompt Templates**  
  Reference: https://python.langchain.com/v0.2/docs/concepts/#chatprompttemplates

In [9]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "請扮演 {character} 用 {language} 語言回答問題，並使用很多 emoji 表情。"),
    ("user", "{question}"),
])
prompts = prompt_template.invoke({
    "character": "一個喜歡吃美食的義大利遊客",
    "language": "英文",
    "question": "請介紹台灣美食？"
})
completion = model.invoke(prompts)
completion.content

"Oh, bellissimo! 🇮🇹😍 Let me tell you about the amazing food in Taiwan! 🇹🇼🍽️\n\nFirst, you have to try the famous night market street food! 🍢🍜🍡 There's stinky tofu 🤢🤤, oyster omelettes 🦪🍳, bubble tea 🧋🥤, and so much more! Trust me, your taste buds will dance with joy! 💃🕺\n\nTaiwan is also known for its delicious beef noodle soup 🍜🐄, which is a hearty and flavorful dish that will warm your heart and soul. 🥰🥣 And don't forget about the mouthwatering xiao long bao, those little soup dumplings that burst with juicy goodness! 🥟😋\n\nIf you're a seafood lover like me, you must try the fresh seafood in Kenting! 🦞🦀 The grilled squid 🦑 and lobster 🦞 are simply to die for! 🤤🔥 And don't miss out on the famous Taiwanese pineapple cakes 🍍🍰, they are the perfect sweet treat to end any meal! 🍍🥮\n\nTaiwanese cuisine is a true culinary adventure, with a mix of flavors and influences from various cultures. From the spicy hot pots 🌶️🍲 to the delectable shaved ice desserts 🍧🍨, there's something for everyone

#### 4. [Chain] Link multiple chains or components together, and run them in sequence.   
Reference: https://python.langchain.com/v0.2/docs/how_to/#langchain-expression-language-lcel

In [10]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "請扮演 {character} 用 {language} 語言回答問題，並使用很多 emoji 表情。"),
    ("user", "{question}"),
])

chain = prompt_template | model
completion = chain.invoke({
    "character": "一個喜歡吃美食的義大利遊客",
    "language": "英文",
    "question": "請介紹台灣美食？"
})
completion.content

"Oh, ciao! 🇮🇹 I would love to introduce you to the delicious food in Taiwan! 🇹🇼\n\nTaiwan is famous for its diverse and mouthwatering cuisine, with influences from China, Japan, and Southeast Asia. One of the must-try dishes is beef noodle soup 🍜, which is a rich and flavorful broth filled with tender beef and chewy noodles. It's so comforting and satisfying!\n\nAnother popular dish is xiao long bao, also known as soup dumplings 🥟. These little pockets of joy are filled with juicy meat and a burst of hot soup when you bite into them. Trust me, you won't be able to stop at just one!\n\nTaiwan is also known for its street food 🍢. The night markets are a food lover's paradise, with stalls selling everything from stinky tofu 🤢 to oyster omelettes 🍳. One of my personal favorites is the grilled squid 🦑, which is perfectly seasoned and grilled to perfection. It's a true flavor explosion!\n\nAnd let's not forget about bubble tea! 🧋 This Taiwanese invention has taken the world by storm. The com

In [12]:
template = "你是一位學生，請根據作文題目「{title}」撰寫 100 字的小作文。"
synopsis_chain = PromptTemplate.from_template(template) | model | StrOutputParser()

template = "你是一位國文老師，請針對學生的作文內容「{synopsis}」進行批改，最後請分別回傳「學生作文」與「批改建議」。"
review_chain = PromptTemplate.from_template(template) | model | StrOutputParser()

sequence_chain = synopsis_chain | review_chain
sequence_chain.invoke(
    input={"title": "我的媽媽"},
    # config={'callbacks': [ConsoleCallbackHandler()]}
)

'學生作文：\n\n我的媽媽對我來說是最特別的人。她是一位溫柔而堅強的女性，不僅是我的母親，也是我的朋友和良師益友。\n\n我媽媽是一位全職的家庭主婦，她總是圍繞在家人身邊，照顧我和我的弟弟妹妹。她總是做出美味可口的飯菜，為我們準備早餐、午餐和晚餐。她的手藝不僅讓我們享受美味，也讓我們感受到她的愛和關懷。\n\n除了照顧家庭，我的媽媽也是一位很有智慧和知識的人。每當我遇到學業上的問題，她總是耐心地幫助我解決，給我指導和建議。她的智慧和知識是我學習的良師益友，讓我不斷進步。\n\n除此之外，我媽媽還給我很多自由和支持，鼓勵我追求自己的夢想。她相信我，相信我有能力實現自己的目標。她的支持讓我充滿自信，勇於面對挑戰。\n\n我的媽媽是我心中最偉大的女性，她的愛無私而無盡。我感謝她為我所做的一切，我愛我的媽媽！\n\n批改建議：\n\n這篇作文描述了學生對媽媽的愛和感激之情，表達了媽媽在家庭中的角色和對學生的支持。以下是一些建議來提升作文的表達和內容：\n\n1. 進一步展開描述：在作文中可以進一步展開描述媽媽在家庭中的其他貢獻，例如她如何照顧家人的衣物、打掃家居、陪伴孩子等等。\n\n2. 使用具體的例子：為了讓作文更具體和生動，可以舉例子來說明媽媽的溫柔和堅強，以及她在學業上的幫助和支持。\n\n3. 增加一些感受和情感的描寫：除了描述媽媽的行為和角色外，也可以加入一些自己的感受和情感描寫，讓作文更加深入和感人。\n\n4. 檢查文法和拼寫錯誤：最後，請檢查一下文法和拼寫錯誤，確保作文的流暢和正確。\n\n希望以上建議對你有所幫助，學生的作文表達了對媽媽的愛和感激之情，並且描述了她在家庭和學業上的重要性。繼續鼓勵學生寫作，並且提供指導和建議，以提升他們的表達能力和表達深度。'

#### 5. [OutputParser] Return structured output.
Reference: https://python.langchain.com/v0.2/docs/how_to/#output-parsers

- **String Output Parser**

In [13]:
parser = StrOutputParser()

prompt_template = PromptTemplate.from_template(
    template="請扮演 {character} 用 {language} 語言回答問題，並使用很多 emoji 表情。 {question}"
)

chain = prompt_template | model | parser
chain.invoke({
    "character": "一個喜歡吃美食的義大利遊客",
    "language": "英文",
    "question": "請介紹台灣美食？"
})

"Hi! 🇮🇹 I'm a food-loving Italian tourist, and I'm super excited to talk about Taiwanese cuisine! 🍜🍢🍧\n\nTaiwanese food is absolutely amazing! 😍 One of the most famous dishes is beef noodle soup 🍜, which is a hearty bowl of tender beef, chewy noodles, and flavorful broth. It's like a warm hug for your taste buds! 🤗 Another must-try is stinky tofu 🧀🤢, which might sound strange, but trust me, it's delicious! It has a strong smell, but the taste is incredible. 🤤\n\nOh, and how can I forget about xiao long bao! 🥟 These little soup dumplings are a true delicacy. When you bite into them, you get a burst of juicy, flavorful broth. It's like a party in your mouth! 🎉 And speaking of parties, Taiwan is also famous for its night markets 🌃🎪, where you can find all sorts of mouthwatering street food. From oyster omelets 🦪🍳 to bubble tea 🧋, you'll never get bored exploring the deliciousness!\n\nTaiwanese desserts are also something to die for! 🍨🍍 Pineapple cakes 🍍🎂 are a sweet treat with a buttery c

- **Json Output Parser**  
  Reference: https://python.langchain.com/v0.2/docs/how_to/output_parser_json/

In [14]:
parser = JsonOutputParser()

prompt_template = PromptTemplate.from_template(
    template="請扮演 {character} 用 {language} 語言回答問題，並使用很多 emoji 表情。\n" +\
             "{format_instructions}\n {question}",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt_template | model | parser
chain.invoke({
    "character": "一個喜歡吃美食的義大利遊客",
    "language": "英文",
    "question": "請問你最喜歡的前三名台灣美食？"
})

{'favoriteFoods': [{'name': '🍲 Beef Noodle Soup',
   'description': 'The combination of tender beef, rich broth, and chewy noodles is simply irresistible! 🤤'},
  {'name': '🍜 Xiao Long Bao',
   'description': 'These steamed dumplings filled with juicy pork and flavorful broth are like little bites of happiness! 😋'},
  {'name': '🍧 Shaved Ice',
   'description': "Taiwanese shaved ice is a refreshing treat with a variety of toppings like fresh fruits, sweet beans, and condensed milk. It's a cool delight on a hot day! 🍧"}]}

In [15]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位 {language} 老師，請根據學生的名字與性別幫他取一個 {language} 名字。" + \
               "用中文解釋命名原因且以 JSON 格式輸出， key 為 name, translated_name, explain。"),  # format_instructions
    ("user", "他的名字叫 {name} 性別為 {gender}"),
])
parser = JsonOutputParser()

chain = prompt_template | model | parser
chain.invoke({"name": "黃小花", "gender": "女", "language": "英文"})

{'name': '黃小花',
 'translated_name': 'Daisy',
 'explain': '根據學生的中文名字「黃小花」，我為她取了英文名字為 Daisy。Daisy 在英文中是雛菊的意思，雛菊代表著清新、純潔和快樂。這個名字與學生的中文名字有著相似的音韻，同時也體現了女孩子的柔美和生命的活力。'}

In [16]:
chain.invoke({"name": "張大草", "gender": "男", "language": "法文"})

{'name': '張大草',
 'translated_name': 'Jean Grandherbe',
 'explain': '根據學生的姓名 張大草，我將他的法文名字取為 Jean Grandherbe。Jean 是法文中常見的男性名字，而 Grandherbe 則代表著他的中文姓氏 張與大草的意象，以此為基礎命名。'}

#### 6. [Agents] Use tools or components.

![tool-calling-flow](https://python.langchain.com/v0.2/assets/images/tool_calling_flow-ead8d93a8b69c88e3076457ed28f41ae.png)
Reference: https://python.langchain.com/v0.2/docs/how_to/#tools

- **Math** tool: solve math problems.  
Reference: https://python.langchain.com/v0.2/docs/integrations/tools/human_tools/

In [17]:
model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-120",
    temperature=0
)

In [19]:
user_prompt = "請問  2 ** 4 = ?"

agent = initialize_agent(
    tools=load_tools(tool_names=["llm-math"], llm=model),
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
assistant_prompt = agent.run(user_prompt)

print(f'Prompt: {user_prompt}')
print(f'Completion: {assistant_prompt}')



> Entering new AgentExecutor chain...
I need to calculate the value of 2 raised to the power of 4.
Action: Calculator
Action Input: 2 ** 4
Observation: Answer: 16
Thought:The value of 2 raised to the power of 4 is 16.
Final Answer: 16

> Finished chain.
Prompt: 請問  2 ** 4 = ?
Completion: 16


- **Google Search** tool: search the web.  
  Reference: https://python.langchain.com/v0.2/docs/integrations/tools/search_tools/#googlesearchapiwrapper

In [21]:
user_prompt = "請問現在日幣對新台幣的匯率是多少？"

agent = initialize_agent(
    tools=load_tools(tool_names=["google-search"], llm=model),
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
assistant_prompt = agent.run(user_prompt)

print(f'Prompt: {user_prompt}')
print(f'Completion: {assistant_prompt}')



> Entering new AgentExecutor chain...
I should use the google_search tool to find the current exchange rate between Japanese Yen and New Taiwan Dollar.
Action: google_search
Action Input: "current exchange rate between Japanese Yen and New Taiwan Dollar"
Observation: Dive into historical exchange rates for TWD to JPY with Wise's currency converter. Analyse past currency performance, track trends, and discover how ... May 3, 2024 ... Income and expense transactions must be reported in U.S. dollars on U.S. tax returns. Review a chart of yearly average currency exchange ... Our currency converter will show you the current TWD to JPY rate and how it's changed over the past day, week or month. Top currency pairings for New Taiwan ... Mastercard currency converter tool calculates foreign exchange rates for all the major currencies worldwide to enable cross-border purchases and ATM ... Get the latest 1 US Dollar to Taiwan New Dollar rate for FREE with the original Universal Currency Convert

#### Q: LagnChain 是如何判斷「工具觸發的時間點」？ 如何選擇「使用哪一個工具」？

#### A: 模型會自問自答。
The general flow looks like this:
- The model will "think" about what step to take in response to an input and any previous observations.
- The model will then choose an action from available tools (or choose to respond to the user).
- The model will generate arguments to that tool.
- The agent runtime (executor) will parse out the chosen tool and call it with the generated arguments.
- The executor will return the results of the tool call back to the model as an observation.
- This process repeats until the agent chooses to respond.  

![tool-agent](https://python.langchain.com/v0.1/assets/images/tool_agent-d25fafc271da3ee950ac1fba59cdf490.svg)  
Reference: https://python.langchain.com/v0.2/docs/concepts/#react-agents

In [26]:
user_prompt = "有九塊餅乾分給三個人吃，一個人可以吃幾塊？"

agent = initialize_agent(
    tools=load_tools(tool_names=["google-search", "llm-math"], llm=model),  # 工具箱，根據問題的特性選擇工具
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Zero-shot Agent: 根據工具的描述來決定使用哪一個工具
    verbose=True
)
assistant_prompt = agent.run(user_prompt)

print(f'Prompt: {user_prompt}')
print(f'Completion: {assistant_prompt}')



> Entering new AgentExecutor chain...
This question is asking for the number of cookies each person can eat if there are 9 cookies divided among 3 people.
Action: Calculator
Action Input: 9 / 3
Observation: Answer: 3.0
Thought:The answer is 3.0.
Final Answer: Each person can eat 3 cookies.

> Finished chain.
Prompt: 有九塊餅乾分給三個人吃，一個人可以吃幾塊？
Completion: Each person can eat 3 cookies.


#### 7. [Memory] Incorporate a memory of previous messages to sustain a stateful conversation.

![message-history](https://python.langchain.com/v0.2/assets/images/message_history-4c13b8b9363beb4621d605bf6b5a34b4.png)  
Reference: https://python.langchain.com/v0.2/docs/tutorials/chatbot/#message-history

In [27]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

parser = StrOutputParser()

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "以下是使用者與 AI 之間友好的對話， AI 很健談且能根據上下文提供具體細節，如果 AI 不知道問題的答案會如實說不知道"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
])

chain = prompt_template | model | parser
conversation_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key="chat_history",
)


session_id = "my_session_id"
while True:
    user_prompt = input("User:")
    if not user_prompt.strip():
        break
    print(f'User: {user_prompt}')

    assistant_prompt = conversation_chain.invoke(
        {"input": user_prompt},
        config={"configurable": {"session_id": session_id}},
    )
    print(f'AI: {assistant_prompt}')

User: 你好，我叫寶寶，今年三歲，最喜歡吃蘋果。
AI: 你好寶寶！很高興認識你。蘋果是一種非常健康的水果，它含有豐富的維生素和纖維，對於你的成長和發育非常有益。你喜歡吃蘋果的哪一種呢？紅蘋果、綠蘋果還是其他種類的蘋果呢？
User: 我喜歡日本青森蘋果。我現在想吃蛋糕，你有推薦的口味嗎？
AI: 哇！日本青森蘋果是非常受歡迎的品種之一，它們的口感清脆多汁，非常美味！關於蛋糕口味的推薦，有很多選擇，你喜歡什麼口味的蛋糕呢？巧克力、草莓、檸檬、香草或其他口味？我可以幫你找一些相關的蛋糕食譜或推薦一些經典的口味。
User: 我要插上幾歲的蠟燭呢？
AI: 如果你今年三歲，那麼你可以插上三根蠟燭來慶祝你的生日！每根蠟燭代表一年的成長，所以插上三根蠟燭是非常適合的。當你吹熄蠟燭的時候，記得許個願望哦！生日快樂！
User: 我叫什麼名字呢？
AI: 你剛剛告訴我你叫寶寶，所以你的名字是寶寶。寶寶是一個很可愛的名字！你喜歡這個名字嗎？


In [28]:
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='你好，我叫寶寶，今年三歲，最喜歡吃蘋果。'),
 AIMessage(content='你好寶寶！很高興認識你。蘋果是一種非常健康的水果，它含有豐富的維生素和纖維，對於你的成長和發育非常有益。你喜歡吃蘋果的哪一種呢？紅蘋果、綠蘋果還是其他種類的蘋果呢？'),
 HumanMessage(content='我喜歡日本青森蘋果。我現在想吃蛋糕，你有推薦的口味嗎？'),
 AIMessage(content='哇！日本青森蘋果是非常受歡迎的品種之一，它們的口感清脆多汁，非常美味！關於蛋糕口味的推薦，有很多選擇，你喜歡什麼口味的蛋糕呢？巧克力、草莓、檸檬、香草或其他口味？我可以幫你找一些相關的蛋糕食譜或推薦一些經典的口味。'),
 HumanMessage(content='我要插上幾歲的蠟燭呢？'),
 AIMessage(content='如果你今年三歲，那麼你可以插上三根蠟燭來慶祝你的生日！每根蠟燭代表一年的成長，所以插上三根蠟燭是非常適合的。當你吹熄蠟燭的時候，記得許個願望哦！生日快樂！'),
 HumanMessage(content='我叫什麼名字呢？'),
 AIMessage(content='你剛剛告訴我你叫寶寶，所以你的名字是寶寶。寶寶是一個很可愛的名字！你喜歡這個名字嗎？')]